In [1]:
import numpy as np
from random import seed
from random import random
from random import randint

#Define dictionary with monsters and their stats.
monsters = {'goblin':{'stats':[8,14,10,10,8,8],'ac':15,'hp':7,'actions':{'scimitar':{'2hit':4,'damage':'1d6+2'},'shortbow':{'2hit':4,'damage':'1d6+2'}}}}

In [2]:
print(monsters['goblin'])
print(monsters['goblin']['stats'][1])
print(monsters['goblin']['actions']['scimitar']['damage'])
#for x in monsters.values():
#  print(x)

{'stats': [8, 14, 10, 10, 8, 8], 'ac': 15, 'hp': 7, 'actions': {'scimitar': {'2hit': 4, 'damage': '1d6+2'}, 'shortbow': {'2hit': 4, 'damage': '1d6+2'}}}
14
1d6+2


In [3]:
#Create an initiative roll function.
def init(combatant):
    #Roll a d20 and add the initiative modifier of the combatant.
    init1 = np.random.randint(1,21) + int((monsters[combatant]['stats'][1] - 10)/2 )
    return init1
    
print('Initiative =',init('goblin'))

#Create a function to determine if one combatant hits the other with their attack.
def hit(combatant1,combatant2):
    #Flag for if the attack hits 0=miss, 1=hit, 2=crit.
    success = 0
    #Roll a d20 and add the hit modifier for the attack.
    roll = np.random.randint(1,21) + int(monsters[combatant1]['actions']['scimitar']['2hit'])
    #Check hit rolled vs AC.
    print('roll to hit =',roll,' vs AC',monsters[combatant2]['ac'])
    if roll >= int(monsters[combatant2]['ac']):
        success = 1
    if roll == 20+int(monsters[combatant1]['actions']['scimitar']['2hit']):
        success = 2
    return success
print('Hit = ',hit('goblin','goblin'))

def damage(combatant):
    #Read in the damage of the weapon.
    damage = monsters[combatant]['actions']['scimitar']['damage']
    
    #Find the location of 'd' in the damage. E.g. 1d6+2 has 'd' at position 1 in damage.
    for i in range(len(damage)):
        #print(damage[i])
        if damage[i]=='d':
            d = i
    
    #Find the location of '+' in damage or see if it doesn't exist (i.e. no modifier).
    sign = 0
    for i in range(len(damage)):
        if damage[i]=='+':
            sign = i
    
    #Find how many dice to roll for the attack damage by reading up to 'd' and setting that value to an integer.
    for i in range(d):
        if i==0:
            num = damage[i]
        else:
            num = num + damage[i]
    #Change num from string to int.
    num = int(num)
    
    #Find what type of dice to roll by reading the characters after d, but before '+'.
    #If no modifier to damage roll sign=0 and read from after 'd' to end of len(damage).
    if sign==0:
        for i in range(d+1,len(damage)):
            if i==(d+1):
                die = damage[i]
            else:
                die = die + damage[i]
    #If there is a modifier to the damage read from after 'd' to '+'.
    else:
        for i in range(d+1,sign):
            if i==(d+1):
                die = damage[i]
            else:
                die = die + damage[i]
    #Change die from string to int.
    die = int(die)
    
    #Determine damage modifier by reading from beyond '+' to end of len(damage).
    if sign == 0:
        mod = 0
    else:
        for i in range(sign+1,len(damage)):
            if i==(sign+1):
                mod = damage[i]
            else:
                mod = mod + damage[i]
    #Change mod from string to int.
    mod = int(mod)
    
    #Initialize damage to zero.
    dmg = 0
    #Calculate damage by rolling 'num'd'die'+'mod'.
    for i in range(num):
        dmg = dmg + np.random.randint(1,die+1)
    dmg = dmg + mod
    
    return dmg

print('Damage =',damage('goblin'))

Initiative = 12
roll to hit = 18  vs AC 15
Hit =  1
Damage = 5


In [33]:
def combat(combatant1,combatant2):
    print(damage('goblin'))
    #Set combatant hps.
    hp1 = int(monsters[combatant1]['hp'])
    print('Combatant1 hp = ',hp1)
    hp2 = int(monsters[combatant2]['hp'])
    print('Combatant2 hp = ',hp2)
    
    #Roll initiative for combatants.
    init1 = init(combatant1)
    print('Combatant 1 initiative = ',init1)
    init2 = init(combatant2)
    print('Combatant 2 initiative = ',init2)
    
    #Higher initiative goes first. If tie flip a coin (could add dex modifier tie breaker later).
    if init1>init2:
        first = combatant1
        second = combatant2
        hp1st = hp1
        hp2nd = hp2
    elif init2>init1:
        first = combatant2
        second = combatant1
        hp1st = hp2
        hp2nd = hp1
    #In case of initiative tie flip coin.
    else:
        flip = np.random.randint(1,3)
        if flip == 1:
            first = combatant1
            second = combatant2
            hp1st = hp1
            hp2nd = hp2
        if flip == 2:
            first = combatant2
            second = combatant1
            hp1st = hp2
            hp2nd = hp1
        
        
    print(first,'goes first.')
        
#    first1 = 1    
#    while hp1 and hp2 > 0:
#        finished = 0
#        if first1 == 1:
#            hit1 = hit(combatant1,combatant2)
#            if hit1 == 1:
#                dmg1 = damage(combatant1)
#                hp2 = hp2 - dmg1
#                print('Combatant1 hits for ',dmg1,'!')
#                print('Combatant1 hp = ',hp1,' Combatant2 hp =',hp2)
#                if hp2 < 0:
#                    finished = 1
#                    break
#            else:
#                print('Combatant1 misses.')
    
    while hp1st and hp2nd > 0:
        finished = 0
        #hit = 0 #Be careful! Setting 'hit' here makes it local to the combat function so the hit function fails inside combat then.
        dmg = 0
        hit1st = hit(first,second)
        if hit1st == 1 or hit1st == 2:
            #dmg = damage(first)
            dmg = damage(first)
            hp2nd = hp2nd - dmg
            print(first,' hits for ',dmg,'!')
            print(first,' hp = ',hp1st,'   ',second,' hp =',hp2nd)
            if hp2nd < 0:
                finished = 1
                break
        else:
            print(first,' misses.')
            
    
combat('goblin','goblin')

3
Combatant1 hp =  7
Combatant2 hp =  7
Combatant 1 initiative =  19
Combatant 2 initiative =  3
goblin goes first.
roll to hit = 16  vs AC 15
goblin  hits for  6 !
goblin  hp =  7     goblin  hp = 1
roll to hit = 22  vs AC 15
goblin  hits for  7 !
goblin  hp =  7     goblin  hp = -6


In [ ]:
hit('goblin','goblin')